In [1]:
from tptools import _init_paths
import argparse
from config import cfg
from config import update_config
from utils.utils import create_logger
import pprint

import torch
import torch.backends.cudnn as cudnn

import models
import dataset
import torchvision.transforms as transforms

from utils.utils import get_model_summary
from torchsummary import summary

/media/abhishek/data/college/thesis/thesis_with_dr_duc/conda_envs/dlpruning5/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def parse_args(args___):
    parser = argparse.ArgumentParser(description='Train keypoints network')
    parser.add_argument('--cfg',
                        help='experiment configure file name',
                        required=True,
                        type=str)


    parser.add_argument('opts',
                        help="Modify config options using the command-line",
                        default=None,
                        nargs=argparse.REMAINDER)

    parser.add_argument('--modelDir',
                    help='model directory',
                    type=str,
                    default='')

    parser.add_argument('--logDir',
                    help='log directory',
                type=str,
                default='')
    parser.add_argument('--dataDir',
                        help='data directory',
                        type=str,
                        default='')
    
    args = parser.parse_args(args___)
    return args

In [3]:
config_file_path = 'experiments/coco/transpose_r/exp2/exp2_step1_prune.yaml'
args = parse_args(['--cfg', config_file_path, 'TEST.USE_GT_BBOX', 'True'])
update_config(cfg, args)

logger, final_output_dir, tb_log_dir = create_logger(cfg, args.cfg, 'valid')

logger.info(pprint.pformat(args))
logger.info(cfg)

cudnn.benchmark = cfg.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg.CUDNN.ENABLED


model = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(
    cfg, is_train=False
)
model.init_weights(cfg.TEST.MODEL_FILE)
model = model.to('cuda:0')

Namespace(cfg='experiments/coco/transpose_r/exp2/exp2_step1_prune.yaml', opts=['TEST.USE_GT_BBOX', 'True'], modelDir='', logDir='', dataDir='')
AUTO_RESUME: True
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  COLOR_RGB: True
  DATASET: coco
  DATA_FORMAT: jpg
  FLIP: True
  HYBRID_JOINTS_TYPE: 
  NUM_JOINTS_HALF_BODY: 8
  PROB_HALF_BODY: 0.3
  ROOT: data/coco/
  ROT_FACTOR: 45
  SCALE_FACTOR: 0.35
  SELECT_DATA: False
  TEST_SET: val2017
  TRAIN_SET: train2017
DATA_DIR: 
DEBUG:
  DEBUG: False
  SAVE_BATCH_IMAGES_GT: True
  SAVE_BATCH_IMAGES_PRED: True
  SAVE_HEATMAPS_GT: True
  SAVE_HEATMAPS_PRED: True
GPUS: (0,)
LOG_DIR: log
LOSS:
  TOPK: 8
  USE_DIFFERENT_JOINTS_WEIGHT: False
  USE_OHKM: False
  USE_TARGET_WEIGHT: True
MODEL:
  ATTENTION_ACTIVATION: relu
  BOTTLENECK_NUM: 0
  DIM_FEEDFORWARD: 1024
  DIM_MODEL: 256
  ENCODER_LAYERS: 4
  EXTRA:
    DECONV_WITH_BIAS: False
    FINAL_CONV_KERNEL: 1
    NUM_DECONV_FILTERS: [256]
    NUM_DECONV_KERNELS: [4]
    

=> creating output/coco/transpose_r_chsel/exp2_step1_prune
=> creating log/coco/transpose_r_chsel/exp2_step1_prune_2024-06-26-17-45


=> loading pretrained model models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth


:: pos_embedding is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: conv1.weight is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: bn1.weight is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: bn1.bias is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: bn1.running_mean is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: bn1.running_var is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: bn1.num_batches_tracked is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: layer1.0.conv1.weight is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: layer1.0.bn1.weight is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth
:: layer1.0.bn1.bias is loaded from models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_

In [4]:
cfg.TEST.MODEL_FILE

'models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth'

In [5]:
summary(model, (3, 256, 192))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 128, 96]           9,408
       BatchNorm2d-2          [-1, 64, 128, 96]             128
              ReLU-3          [-1, 64, 128, 96]               0
         MaxPool2d-4           [-1, 64, 64, 48]               0
            Conv2d-5           [-1, 64, 64, 48]           4,096
       BatchNorm2d-6           [-1, 64, 64, 48]             128
  ChannelSelection-7           [-1, 64, 64, 48]               0
              ReLU-8           [-1, 64, 64, 48]               0
            Conv2d-9           [-1, 64, 64, 48]          36,864
      BatchNorm2d-10           [-1, 64, 64, 48]             128
             ReLU-11           [-1, 64, 64, 48]               0
           Conv2d-12          [-1, 256, 64, 48]          16,384
      BatchNorm2d-13          [-1, 256, 64, 48]             512
           Conv2d-14          [-1, 256,